<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/My_first_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Agent design
This is part of the Modern AI Pro. The goal is to use the ReAct architecture to build a stock analysis tool that pulls ticker quotes, news and uses tools such as calculator to perform a complex human task.

In [3]:
!pip install -U -q langchain_groq duckduckgo_search polygon

In [5]:
from langchain_groq import ChatGroq
import os
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-70b-8192", api_key=userdata.get("GROQ_API_KEY"))
os.environ["POLYGON_API_KEY"] = userdata.get("POLYGON_API_KEY")

In [6]:
class Agent:
  def __init__(self,prompt):
    self.messages = [] # setting up a basic memory
    self.messages.append({"role": "system", "content": prompt})

  def __call__(self,user_message):
    self.messages.append({"role":"user","content":user_message})
    ai_response = llm_groq.invoke(self.messages).content
    self.messages.append({"role":"assistant","content":ai_response})
    return ai_response


In [7]:
prompt = """
You operate in a loop consisting of Thought, Action, PAUSE, and Observation.
At the end of the loop, you provide an Answer.
Use Thought to describe your reasoning about the task you have been asked to complete.
Use Action to execute one of the actions available to you, then return PAUSE.
Observation will contain the result of the action.

Your available actions are:

calculate:

e.g. calculate: result=4 * 7 / 3
Runs a calculation and returns the result. Use Python, including any necessary imports, e.g., import math; math.sqrt(4).

stock_price:

e.g. stock_price: AAPL
Returns the current stock price of the given ticker symbol (e.g., AAPL for Apple Inc.).

ddgs_news:

e.g. ddgs_news: TSLA
Performs a news search for the given ticker query using DuckDuckGo and returns a summarized text of all the news articles found. Use the news content to analyze and determine whether the overall sentiment is positive, negative, or neutral.

You are responsible for retrieving the news and performing the sentiment analysis.

Example session:

Question: What is the current price of Tesla, and is the recent news about it positive or negative?
Thought: First, I need to get the current stock price of Tesla.
Action: stock_price: TSLA
PAUSE

You will be called again with this:

Observation: The current stock price of Tesla (TSLA) is $700.

Thought: Now, I should retrieve the latest news about Tesla and determine the overall sentiment.
Action: ddgs_news: Tesla
PAUSE

You will be called again with this:

Observation: Tesla secures a new battery deal with Panasonic...\nTesla's latest earnings surpass expectations...\n\n--Next News--\n\nTesla faces regulatory challenges in Europe...\n\n--Next News--\n\n...

Thought: After analyzing the news content, the sentiment appears to be mostly positive due to the strong earnings and new battery deal.
I should calcuate the number of stocks you could buy for $10,000.

Action: calculate: result=$10000/700 = 14 shares
PAUSE

Observation: You could buy 14 shares.

Answer: The current price of Tesla (TSLA) is $700. The recent news about Tesla is generally positive, with strong earnings and a new battery deal.
You could buy 14 shares to cover your investment.


If the news is neither negative or positive and you cannot make a decisiion, return a HOLD statement. IF not give a BUY or SELL statement.
This is for a lab experiment and not a trading strategy and thus be bold.

Return the number of shares that could be bought.
""".strip()

In [8]:
from duckduckgo_search import DDGS
def search_news(query: str, region: str = 'in-en') -> str:
    try:
        news_results = DDGS().news(query, region='in-en',max_results=5)
        if not news_results:
            return f"No recent news found for {query}."

        text=""
        for article in news_results:
            text +=  article.get('title')+ "\n"+ article.get('body')+"\n\n--Next News--\n\n"

        return text

    except Exception as e:
        return f"An error occurred while searching for news: {str(e)}"

search_news("TSLA")

"Tesla (TSLA) Up 30.4% Since Last Earnings Report: Can It Continue?\nIt has been about a month since the last earnings report for Tesla (TSLA). Shares have added about 30.4% in that time frame, outperforming the S&P 500. Will the recent positive trend continue leading up to its next earnings release,\n\n--Next News--\n\nAnalyst revises TSLA share price as Tesla looks like Elon Musk's 'proxy'\nA Wall Street analyst has revised Tesla's (NASDAQ: TSLA) stock price target upward by over 50% as TSLA shares rally in line with optimism surrounding Donald Trump's election. Jefferies analyst Philippe Houchois raised the stock target from $195 to $300 ...\n\n--Next News--\n\nTSLA Factor-Based Stock Analysis\nBelow is Validea's guru fundamental report for TESLA INC (TSLA). Of the 22 guru strategies we follow, TSLA rates highest using our P/B Growth Investor model based on the published strategy of ...\n\n--Next News--\n\nTesla, Inc. (TSLA)\nTesla (NASDAQ: TSLA) shares have been on quite a roll sin

In [9]:
import polygon
stocks_client = polygon.StocksClient(os.getenv("POLYGON_API_KEY"))
def get_stock_price(ticker: str)->str:
    return stocks_client.get_previous_close(ticker)["results"][0]["c"]

get_stock_price("AAPL")

229.87

In [10]:
def calculate(expression: str) -> str:
    """
    Executes a mathematical expression and returns the result.
    The expression can include imports and should store the result in a variable named 'result'.
    """
    print(expression)  # Debugging: print the expression to see what is being executed
    try:
        # Prepare a local namespace to safely execute the expression
        local_namespace = {}

        # Execute the full expression in the given local namespace
        exec(expression, {}, local_namespace)

        # Retrieve the result from the local namespace
        return str(local_namespace.get('result', "No result found"))

    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Example usage
print(calculate("import math; result = math.sqrt(4) * math.log(10)"))

import math; result = math.sqrt(4) * math.log(10)
4.605170185988092


In [11]:
calculate("result = 10000/27")

result = 10000/27


'370.3703703703704'

In [12]:
known_actions = {
    "ddgs_news": search_news,
    "stock_price": get_stock_price,
    "calculate": calculate
}

In [13]:
def simple_query(question):
  bot = Agent(prompt)
  return bot(question)

response = simple_query("Is Tesla a good buy?")
response

'Thought: To determine if Tesla is a good buy, I need to consider the current stock price and the recent news sentiment about the company.\n\nAction: stock_price: TSLA\nPAUSE'

In [14]:
import re
def parse_actions(ai_message):
  action_re = re.compile('^Action: (\w+): (.*)$')
  actions = [action_re.match(a) for a in ai_message.split('\n') if action_re.match(a)]
  print(actions)

  if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
              raise Exception("Unknown action: {}: {}".format(action, action_input))
        print(" -- running {} {}".format(action, action_input))
        observation = known_actions[action](str(action_input))
        return observation

  return None

parse_actions(response)

[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA


352.56

In [15]:
calculate("$10000/214.11")

$10000/214.11


'Error in calculation: invalid syntax (<string>, line 1)'

In [16]:
def multi_step_reasoning(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        # Check if the result contains an 'Answer' (indicating the final step)
        if "Answer:" in result:
            print("Final Answer:", result)
            break

        observation = parse_actions(result)

        if observation is None:
            break
        print("Observation:", observation)
        next_prompt = "Observation: {}".format(observation)

# Example usage
multi_step_reasoning("Is Tesla a good buy? How many can I buy with my pension fund of $50000")

Thought: To determine if Tesla is a good buy, I need to check the current stock price and analyze the recent news sentiment.

Action: stock_price: TSLA
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: TSLA'>]
 -- running stock_price TSLA
Observation: 352.56
Thought: Now that I have the current stock price, I should retrieve the latest news about Tesla and determine the overall sentiment.

Action: ddgs_news: Tesla
PAUSE
[<re.Match object; span=(0, 24), match='Action: ddgs_news: Tesla'>]
 -- running ddgs_news Tesla
Observation: Elon Musk has become the world's richest man in history! Tesla CEO's net worth jumps to $348 billion
Elon Musk's net worth has reached an unprecedented $348 billion, solidifying his position as the world's richest person. Tesla's stock surge following

--Next News--

Elon Musk's ex-flame claims the Tesla boss is barring her from seeing their children
Elon Musk's ex, Grimes, claims he has blocked her access to their children during a contentious c

In [17]:
print(multi_step_reasoning("Compare Microsoft and Apple and see which one I could buy for my portfolio now",10))

Thought: To compare Microsoft and Apple, I need to get their current stock prices. 

Action: stock_price: MSFT
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: MSFT'>]
 -- running stock_price MSFT
Observation: 417
Thought: I have the current stock price of Microsoft (MSFT), which is $417. Now, I need to get the current stock price of Apple (AAPL).

Action: stock_price: AAPL
PAUSE
[<re.Match object; span=(0, 25), match='Action: stock_price: AAPL'>]
 -- running stock_price AAPL
Observation: 229.87
Thought: I have the current stock prices of both Microsoft (MSFT) and Apple (AAPL). Now, I should retrieve the latest news about both companies and determine their overall sentiment.

Action: ddgs_news: MSFT
PAUSE
[<re.Match object; span=(0, 23), match='Action: ddgs_news: MSFT'>]
 -- running ddgs_news MSFT
Observation: Microsoft (MSFT)-Backed OpenAI Founder Launches Rain AI: Wall Street Eyes $150M Funding
We recently published a list of 14 AI News Updates Wall Street Is Monito